#### readme: 
* reference: 
[在Keras模型中使用预训练的词向量](https://keras-cn-docs.readthedocs.io/zh_CN/latest/blog/word_embedding/)
* summary: 
    - method: use pretrained word2vec 百度百科 + CNN
    - updated: use randomly initialized for word embedding matrix, instead of all 0 => [reference](https://towardsdatascience.com/word2vec-from-scratch-with-numpy-8786ddd49e72)
        - reference: 
            - [medium article](https://towardsdatascience.com/machine-learning-multiclass-classification-with-imbalanced-data-set-29f6a177c1a) + [github code](https://github.com/javaidnabi31/Multi-class-with-imbalanced-dataset-classification/blob/master/20-news-group-classification.ipynb)
            

In [2]:
import os
import numpy as np
import pandas as pd
os.chdir('/Users/liyuan/desktop/CSAir/codes/word2vec_pretrained')

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Flatten
from keras.layers import Embedding
import tensorflow as tf

from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from pandas_ml import ConfusionMatrix

import warnings
warnings.simplefilter('ignore')

from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from pandas_ml import ConfusionMatrix

from word2vec_v2 import word2vec

In [4]:
w2v = word2vec()
embeddings_index = w2v.load_pretrained_vectors('../../Source_Data/sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5')
# X_train, y_train, X_val, y_val = w2v.prepare_data('./res/all_labeled_data_v3.csv')
X_train, y_train, X_val, y_val = w2v.prepare_data('../../res/labeled_data_with_without_tk.csv')
# train data 
predicted_label_list = w2v.train_data(X_train,y_train,X_val,y_val)

Found 635922 word vectors.
Found 4747 unique tokens.
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551,)
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551, 10)
Train on 1241 samples, validate on 310 samples
Epoch 1/10
1241/1241 [==============================] - 32s 26ms/step - loss: 2.1521 - acc: 0.2127 - val_loss: 2.1079 - val_acc: 0.2226
Epoch 2/10
1241/1241 [==============================] - 31s 25ms/step - loss: 1.9544 - acc: 0.2611 - val_loss: 1.9381 - val_acc: 0.3452
Epoch 3/10
1241/1241 [==============================] - 31s 25ms/step - loss: 1.7365 - acc: 0.3989 - val_loss: 1.8495 - val_acc: 0.3645
Epoch 4/10
1241/1241 [==============================] - 29s 24ms/step - loss: 1.4891 - acc: 0.4980 - val_loss: 1.7497 - val_acc: 0.3774
Epoch 5/10
1241/1241 [==============================] - 31s 25ms/step - loss: 1.3939 - acc: 0.5165 - val_loss: 1.6889 - val_acc: 0.3903
Epoch 6/10
1241/1241 [==============================] - 30s 24ms/step - loss: 1.2608 

In [5]:
# check label index
print('label encoding dictionary:', w2v.labels_index)

label encoding dictionary: {'计划': 7, '机上': 5, '中转': 0, '售后': 3, '预订': 9, '设计': 8, '出发': 1, '性能': 4, '行程': 6, '到达': 2}


In [6]:
val_df = w2v.incorporate_pred_label()
val_df = w2v.map_label(val_df,predicted_label_list)
val_df.head()

310


,review,review_tokens,label,label_encoded,pred_label_encodes,pred_label
414,"这是最烂的一次旅行！广州地勤垃圾的不能再垃圾！8:20的航班延误到9:20,然后在登机口等到...",这是 最烂 旅行 广州 地勤 垃圾 垃圾 航班 延误 登机口 点多 安排 住宿 折腾 酒店 ...,中转,0,0,中转
1092,取消航班也不通知，南航服务太差,取消 航班 通知 南航 服务 太 差,出发,1,1,出发
719,查询航班买机票很方便，好用,查询 航班 买 机票 好用,预订,9,9,预订
1430,行李箱明显是撞击导致损坏，航空公司当然赔偿我一个行李箱。可是行李箱质量跟我自己差太多。当时已...,行李箱 撞击 导致 损坏 航空公司 赔偿 行李箱 行李箱 质量 差太多 接受 补偿 行李箱 ...,到达,2,2,到达
613,优惠券用不了,优惠券,预订,9,4,性能


In [7]:
# evaluate performance
w2v.evaluate_performance(val_df)

Confusion Matrix:

Predicted  中转   出发  到达  售后  性能  机上  行程  计划  设计  预订  __all__
Actual                                                     
中转         17    9   0   0   2   1   0   0   0   0       29
出发          7   43   6   5   0   1   3   0   0   4       69
到达          3   14  10   0   0   1   0   0   0   0       28
售后          1    7   0  11   2   0   0   0   0   3       24
性能          0    8   0   3  13   1   0   0   0   8       33
机上          6   12   2   0   3  25   0   0   0   1       49
行程          2    4   0   1   2   0   1   0   0   3       13
计划          0    2   1   0   3   0   0   0   0   0        6
设计          0    1   0   0  10   0   0   0   0   5       16
预订          0   11   0   1   6   0   2   0   0  23       43
__all__    36  111  19  21  41  29   6   0   0  47      310


Overall Statistics:

Accuracy: 0.4612903225806452
95% CI: (0.4047890057286, 0.5185383637691552)
No Information Rate: ToDo
P-Value [Acc > NIR]: 0.00011915078548962167
Kappa: 0.3646917337522089
Mcnemar

In [8]:
# output => probability of each review concerning each class label
output = w2v.output
# construct a result matrix, based on the output matrix (contain probability info)
# pick the index of class(with highest proba) for each review
picked_label_indices = [w2v.output[i].argmax(axis=-1) for i in range(len(output))]
# print('labels picked for each review:', picked_label_indices)

# initialize a res list
res = []
for i in range(len(output)):
    # initialize a list with 0, e.g., [0,0,...0]
    init = [0] * output.shape[1]
    # set the picked label index to be 1, while the rest as 0
    init[picked_label_indices[i]] = 1
    res.append(init)
    
# prepare y_val
y_val_pred = np.array(res)

# plot ROC_AUC
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, y_val_pred)
print('ROC_AUC score:', score)

ROC_AUC score: 0.6450518571341632


In [10]:
# implement oversampling
predicted_label_list_os = w2v.over_sampling()
val_df = w2v.incorporate_pred_label()
val_df_os = w2v.map_label(val_df,predicted_label_list_os)
val_df_os.head()

(1478, 1000) (1478, 10)


NameError: name 'X_val' is not defined

In [92]:
# evaluate performance after over sampling
w2v.evaluate_performance(val_df_os)

Confusion Matrix:

Predicted  中转  出发  到达  售后  性能  机上  行程  计划  设计  预订  __all__
Actual                                                    
中转          3   5   0   2   0   3   0  10   0   3       26
出发          5  16  10   4   1   6   0  19   0  14       75
到达          0   2  11   0   0   4   0  17   0   0       34
售后          0   2   0  10   3   1   1   2   0   8       27
性能          0   1   0   0   8   1   0   3   0   9       22
机上          0   4   4   1   2  34   1   9   0   1       56
行程          0   0   0   0   3   0   0   2   0   8       13
计划          0   0   0   0   3   0   0   5   0   1        9
设计          0   0   0   0   3   0   0   0   1   1        5
预订          0   2   0   2   9   0   0   7   0  23       43
__all__     8  32  25  19  32  49   2  74   1  68      310


Overall Statistics:

Accuracy: 0.3580645161290323
95% CI: (0.304666114521708, 0.4142047918295947)
No Information Rate: ToDo
P-Value [Acc > NIR]: 1.6248443439288728e-06
Kappa: 0.27474723724429817
Mcnemar's Test P-

summary: oversampling vs. non-oversampling has very similar result